### Basic import

In [16]:
import rosbag
#import roslib
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge, CvBridgeError
#from rospy import Subscriber
#from sensor_msgs.msg import Image
#from opt_msgs.msg import DetectionArray
#import message_filters
#from message_filters import TimeSynchronizer, Subscriber
#import socket, json
import re
from detectionExtraction import *

In [17]:
# Create bag to process
bag = rosbag.Bag('../../bags/quality.bag')
print bag

path:        ../../bags/quality.bag
version:     2.0
duration:    29.8s
start:       May 23 2017 17:15:57.03 (1495552557.03)
end:         May 23 2017 17:16:26.82 (1495552586.82)
size:        2.0 GB
messages:    992
compression: none [338/338 chunks]
types:       opt_msgs/DetectionArray [2be460a0a39ff0f809695e59da9d469e]
             opt_msgs/TrackArray     [4106b2da5c66b9f239aada7912e67b8f]
             sensor_msgs/Image       [060021388200f6f0f447d0fcd9c64743]
topics:      /detector/detections      328 msgs @ 12.5 Hz : opt_msgs/DetectionArray
             /kinect2_head/rgb/image   337 msgs @ 11.3 Hz : sensor_msgs/Image      
             /tracker/tracks           327 msgs @ 13.7 Hz : opt_msgs/TrackArray


In [18]:
# create bridge between CV and ROS
br = CvBridge()

In [19]:
# Create array for rgb frames
rgbTime = []
rgbSeq = []
for topic, msg, t in bag.read_messages(topics=['/kinect2_head/rgb/image']):
    time = msg.header.stamp
    rgbTime.append(int(str(time)))
    seq = msg.header.seq
    rgbSeq.append(seq)
    msg.encoding = "bgr8"
    cv_image = br.imgmsg_to_cv2(msg, desired_encoding="bgr8")
    cv2.imwrite('/home/sabrine/notebook/framesRGB/' + str(seq) + '.jpg',cv_image)

In [20]:
detect = detection_extraction('../../bags/quality.bag')

In [21]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [22]:
match_frame= []
for index, row in detect.iterrows():
    #print index
    rgbSeqMatched = find_nearest(np.asarray(rgbTime), int(str(row.timestamp)))
    match_frame.append(rgbSeqMatched)

In [23]:
match_frame

[1495552557206957876,
 1495552557206957876,
 1495552557260235570,
 1495552557399768317,
 1495552557594792111,
 1495552557752511223,
 1495552557872844529,
 1495552557872844529,
 1495552557983076042,
 1495552558050087280,
 1495552558050087280,
 1495552558118595988,
 1495552558198290814,
 1495552558297059942,
 1495552558368512321,
 1495552558443158773,
 1495552558514840698,
 1495552558595797186,
 1495552558681005600,
 1495552558811344761,
 1495552558926397490,
 1495552558926397490,
 1495552558926397490,
 1495552559014812321,
 1495552559014812321,
 1495552559082376413,
 1495552559082376413,
 1495552559082376413,
 1495552559170681813,
 1495552559170681813,
 1495552559266965900,
 1495552559266965900,
 1495552559360642087,
 1495552559360642087,
 1495552559489166196,
 1495552559489166196,
 1495552559554742182,
 1495552559554742182,
 1495552559617382081,
 1495552559617382081,
 1495552559690420364,
 1495552559690420364,
 1495552559832642880,
 1495552559832642880,
 1495552559921862297,
 149555255

In [27]:
len(match_frame)

847

In [28]:
detect['closestFrameTime'] = match_frame

In [29]:
detect

,timestamp,seq,detectionCoordinates,height,distance,closestFrameTime
0,1495552557206957000,5080,box_2D: \n x: 354\n y: 158\n wi...,2.19331693649,8.11842536926,1495552557206957876
1,1495552557206957000,5080,box_2D: \n x: 30\n y: 200\n wid...,1.3962341547,7.84061908722,1495552557206957876
2,1495552557260235000,5081,box_2D: \n x: 371\n y: 154\n wi...,2.22669267654,7.89670038223,1495552557260235570
3,1495552557399768000,5082,box_2D: \n x: 113\n y: 204\n wi...,1.30548119545,7.20469284058,1495552557399768317
4,1495552557594792000,5084,box_2D: \n x: -39\n y: 172\n wi...,1.5031799078,3.84624624252,1495552557594792111
5,1495552557752511000,5086,box_2D: \n x: -49\n y: 141\n wi...,1.73955285549,3.80243611336,1495552557752511223
6,1495552557872844000,5087,box_2D: \n x: 372\n y: 154\n wi...,2.2212126255,7.89465427399,1495552557872844529
7,1495552557872844000,5087,box_2D: \n x: -43\n y: 141\n wi...,1.74076521397,3.76684641838,1495552557872844529
8,1495552557983076000,5088,box_2D: \n x: -37\n y: 140\n wi...,1.749984622,3.7343711853,1495552557983076042
9,1495552558050087000,5089,box_2D: \n x: -30\n y: 137\n wi...,1.77359616756,3.70270895958,1495552558050087280


In [35]:
data = np.asarray([np.asarray(rgbTime), np.asarray(rgbSeq)]).T

In [38]:
rgb = pd.DataFrame(data=data)

In [42]:
rgb.columns = ['rgbTime', 'rgbSeq']

In [44]:
synchronize = pd.merge(detect, rgb, how='inner', left_on='closestFrameTime', right_on='rgbTime',
         left_index=False, right_index=False, sort=True,
         suffixes=('_detect', '_rgb'), copy=True, indicator=False)

In [45]:
synchronize

,timestamp,seq,detectionCoordinates,height,distance,closestFrameTime,rgbTime,rgbSeq
0,1495552557206957000,5080,box_2D: \n x: 354\n y: 158\n wi...,2.19331693649,8.11842536926,1495552557206957876,1495552557206957876,2
1,1495552557206957000,5080,box_2D: \n x: 30\n y: 200\n wid...,1.3962341547,7.84061908722,1495552557206957876,1495552557206957876,2
2,1495552557260235000,5081,box_2D: \n x: 371\n y: 154\n wi...,2.22669267654,7.89670038223,1495552557260235570,1495552557260235570,3
3,1495552557399768000,5082,box_2D: \n x: 113\n y: 204\n wi...,1.30548119545,7.20469284058,1495552557399768317,1495552557399768317,4
4,1495552557594792000,5084,box_2D: \n x: -39\n y: 172\n wi...,1.5031799078,3.84624624252,1495552557594792111,1495552557594792111,6
5,1495552557752511000,5086,box_2D: \n x: -49\n y: 141\n wi...,1.73955285549,3.80243611336,1495552557752511223,1495552557752511223,8
6,1495552557872844000,5087,box_2D: \n x: 372\n y: 154\n wi...,2.2212126255,7.89465427399,1495552557872844529,1495552557872844529,9
7,1495552557872844000,5087,box_2D: \n x: -43\n y: 141\n wi...,1.74076521397,3.76684641838,1495552557872844529,1495552557872844529,9
8,1495552557983076000,5088,box_2D: \n x: -37\n y: 140\n wi...,1.749984622,3.7343711853,1495552557983076042,1495552557983076042,10
9,1495552558050087000,5089,box_2D: \n x: -30\n y: 137\n wi...,1.77359616756,3.70270895958,1495552558050087280,1495552558050087280,11
